In [ ]:
# encoding:utf-8
"""
作者：王若愚
工作：1.合并文章查询程序与获取文章内容程序
     2.添加sql语句
时间：2018/05/01 03:16
"""
import scrapy
from bs4 import BeautifulSoup
import time
import sched
import queue
#import Queue
import requests
import json
import pymysql

# 设定间隔时间 秒
inc = 5

class main(object):

    def __init__(self):
        self.url = 'https://www.ihuoqiu.com/MAPI/GetArticleListData'  # 首页网址url
        #self.urlqueue = Queue.Queue() # python2 创建url的队列
        self.urlqueue = queue.Queue()  #python3 创建url的队列
        
        # 模拟成浏览器
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.109 Safari/537.36',
            'Referer': 'http://m.ihuoqiu.com/index',
            'Origin': 'http://m.ihuoqiu.com',
            'Connection':'keep-alive',
            'Content-Type':'application/json;charset=utf-8',
        }
        
        #设定schedule对象
        self.schedule = sched.scheduler(time.time, time.sleep) 

        self.article = []  # 存放标题的变量
        self.content = {}    # 存放文章内容
    
    def perform_command(self,inc):
        self.schedule.enter(inc,0,self.perform_command,(inc,))
        self.run_()
    def timming_exe(self,inc = inc):
        #enter 用来安排事件的发生
        self.schedule.enter(inc,0,self.perform_command,(inc,))
        self.schedule.run()
        
    def run_(self):
        
        # 该线程要执行的任务,即获得url队列,传入页面的url获得页面的代码
        response = None
        content = ''
        """
        # 使用代理服务器访问网页
        proxy_addr = {
            "http": "http://127.0.0.1:8080",
            "https": "https://127.0.0.1:8080"
                      }
        """
        # 构建请求的request
        post_data = '''{"Type":1,"PageIndex":1,"PageSize":1000}'''
        #r = requests.post(url=self.url, data=post_data, headers=self.headers, proxies=proxy_addr, verify=False)
        r = requests.post(url=self.url, data=post_data, headers=self.headers)

        # 从页面代码中获取title的链接
        if r.status_code == 200:
            # 获得页面代码
            data = r.content
            j_data = json.loads(data)
            #print (j_data)

            #将页面转化为json格式
            if "code" in j_data.keys() and j_data.get("code") == 200:
                all_articles = j_data.get("data", [])
                all_articles_list = all_articles[2:-2].split("},{")
                ll = []
                for one in all_articles_list:
                    if one[0] == u"{" and one[-1] != u"}":
                        one = one + u"}"
                    elif one[0] != u"{" and one[-1] != u"}":
                        one = u"{" + one + u"}"
                    elif one[0] != u"{" and one[-1] == u"}":
                        one = u"{" + one
                    one = json.loads(one)
                    ll.append(one)
                    
                for article_info in ll:
                    ArticleInfo = article_info.get("ArticleInfo", "No finding the information")
                    #获取页面链接
                    Url_data = article_info.get("data1", None)
                    #获取文章内容
                    article_content = self.get_article_content(Url_data)
                    Title  = ArticleInfo.get("Title", "No finding the content")
                    ImgUrl = ArticleInfo.get("ImgUrl","No finding the img")
                    Source = ArticleInfo.get("Source","No finding the source")
                    Author = ArticleInfo.get("Author","No finding the author")
                    ShortDescription = ArticleInfo.get("ShortDescription", "No finding the description")
                    
                    #将文章数据保存为json格式
                    self.content = {
                        "Title":Title,
                        "ImgUrl":ImgUrl,
                        "Source":Source,
                        "Author":Author,
                        "Url_data":Url_data,
                        "ShortDescription":ShortDescription,
                        "Content":article_content
                    }
                    self.article.append(self.content)
                    
            #show the list of article
            print("The length is:",len(self.article))
            #展示采集数据
            for i in self.article:
                print("the article is:",i)
            """
            准备导入数据库
            """
            self.import_to_db(self.article)
        else:
            print(u"页面加载失败")
            time.sleep(100)
            self.run()
            return None
    #将文章数据导入到mysql中
    def import_to_db(self,art):
        #创建 connection链接
        connection = pymysql.connect(host='mysql-wang.cekkdnv29igt.us-west-2.rds.amazonaws.com',
                       port=3306,user='scrapy',passwd='scrapy25806',charset='utf8')
        try:
            with connection.cursor() as cursor:
                sql = "CREATE database if not exists LWL"
                cursor.execute(sql)
                cursor.execute("use LWL")
                #在该数据库中创建一个名为article的表
                sql = "CREATE TABLE if not exists article(Title VARCHAR(200),Author VARCHAR(20),Url_data VARCHAR(20),Content text)"
                cursor.execute(sql)
                sql = "INSERT INTO article(Title,Author,Content) VALUES(%s,%s,%s,%s)"
                try:
                    #开始插入数据
                    for db_art in art:
                        cursor.execute(sql,(db_art.get("Title", None),db_art.get("Author", None),db_art.get("Url_data",None),db_art.get("Content",None)))
                    connection.commit()
                except:
                    connection.rollback()
                """
                增加一个去重功能
                """
        finally:
            connection.close()
    def get_article_content(self,Url_data):
        url = "https://www.ihuoqiu.com/MAPI/GetArticleInfoData"
        headers = {
            "User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36",
            "Connection":"keep-alive",
            "Content-Type":"application/json;charset=UTF-8",
            "Origin":"http://m.ihuoqiu.com",
            "Referer":"http://m.ihuoqiu.com/article?id="+Url_data+"&type=1"
        }
        data = '''{Type: "1", data: "%s"}'''%Url_data
        response =requests.post(url=url,data=data,headers=headers)
        article1 = []
        
        if response.status_code == 200:
            #将文章内容转化为正常格式
            article_json = json.loads(response.content)
            article_content = article_json.get("data",[])
            try:
                art_content_list = article_content.split(',"')
                article = art_content_list[2].split("。<")
                for i in article:
                    article1.append(BeautifulSoup(i,'lxml').get_text())
                return article1
            except:
                print("The content is None")
                return None

if __name__ == "__main__":
    tit = main()
try:    
    tit.timming_exe(120)
except ConnectionError as e:
    print(e)

In [5]:
# encoding:utf-8
'''
作者：王若愚
功能：
     1.获取文章的页面链接
     2.保存文章内容
     3.保存文章图片
注意事项：
     1.爬取页面的内容可能隐藏与ajax之中
     2.爬取开始之前需要查看network之中的response工具
     3.创建的文件名字结尾不得有标点符号
     4.创建的文件名字中不得出现冒号
记录：
     1.正则表达式在匹配时：(),[],{}的区别
     2.部分未加<p></p>标签的段落，无法完整提取出来
     3.不必加过多的try except语句
     4.再次启动程序时，程序有时无法正常覆盖已有的爬取内容，
       因此，若需重启程序时，建议手动更改原有保存路径或直接将
       原纪录删除
       
时间：2018/05/01
'''
from urllib.request import urlretrieve
import requests
import json
import re
import os
from bs4 import BeautifulSoup


class main(object):
    def __init__(self):
        self.baseurl = "https://www.7234.cn"        
        #设置文件保存路径
        self.path = "H:\Desktop\scrapy"
        self.headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36',
            'Connection':'keep-alive',
            'Accept-Language': 'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
            'Host': 'www.7234.cn'
        }
        
    
    #设置爬取的起始页面号码
    def run(self,start_number=1,end_number=2):
        for i in range(start_number,end_number+1,1):
            post_data = {"page": i}#数字代表页数，通过改变数字爬取其他内容
            print("scraping the %d page..."%i)
            url = "https://www.7234.cn/fetch_articles/news?page=%s"%(post_data["page"])
            #开始抓取每一个页面中出现的文章链接
            self.get_article_link(url)
    
    def get_article_link(self,url):
        artset = set()#保存文章链接的集合除重
        #获取页面内容
        res = requests.get(url)
        if res.status_code == 200:
            #将页面转化为json格式并获取html内容
            data = json.loads(res.content)
            html_json = data.get('html')
            #print(html_json)
            
            #获取文章后缀匹配任意字符
            artId = re.compile("href=(.*?) target")
            result1 = artId.findall(html_json)
            #匹配类似于/news/53805
            for i in result1:
                number = re.compile(r'/\w*/\d+')
                try:
                    Id = re.search(number,i).group()
                except AttributeError:
                    Id = None
                if Id != None:
                    artset.add(self.baseurl+Id)
            #保存文章链接
            artlist = list(i for i in artset)
            print(artlist)
            self.get_article_content(artlist) 
            #print("You have gotten all the page...")
            
            
    def get_article_content(self,artlist):
        #将工作路径赋值给path
        path = self.path
        #设定当前工作路径
        if not os.path.exists(path):
            try:
                os.mkdir(path)
            except:
                print("This path is not valide",path)
        
        for art in artlist:
            print("the link is:",art)
            article_html = requests.get(url = art,headers = self.headers)
            bs0bj = BeautifulSoup(article_html.text,'lxml')
            #获取文章的标题
            try:
                title = bs0bj.find('div',class_="article-inner").find("h1").get_text()
            except:
                title = "The article is None"

            #设定文章所在文件夹的名字
            #注意文件夹名字不得用问号结尾且不得出现冒号
            pattern = re.compile(r'[\!,\?,\:,\：,\|]')
            title = re.sub(pattern,"_",title)
            path = path+"\\"+title
            #创建文章文件夹用以保存文章内容的txt文件和图片
            if not os.path.exists(path):
                try:
                    os.mkdir(path)
                except:
                    print("This path is not valide",path)
                
            #文章内容链表
            para_content = []
            #文章图片链表
            photo_link = []

            #获取文章的内容
            content = bs0bj.find("div",{"class":"a-content"})
            try:
                paragraph = content.find_all("p")
                #分别对每个自然段进行分析
                for p in paragraph:
                    #获取文章中的图片
                    para_content.append(p.get_text())
                    try:
                        ph = "https:"+p.find("img")["src"]
                    except:
                        ph = None
                    if ph != None:
                        photo_link.append(ph)
            except:
                paragraph = "The content is none"

            #print("The title is:",title)
            #print("The content is:",para_content)
            try:
                f = open(path+"\\"+title+".txt","w+",encoding='utf8')
                for i in para_content:
                    f.write(i+'\n')
            except:
                print("The file is not open")
            for i in range(len(photo_link)):
                #try:
                #print("The path of photo is: "+path+"\\"+str(i)+".jpg")
                urlretrieve (photo_link[i], path+"\\"+str(i)+".jpg")
                #except:
                    #print("The photo is not retrieved, "+photo_link[i]+" The link is: "+art)
                    #print("The path of photo is: "+path+"\\"+str(i)+".jpg")
            f.close()
            path = self.path



if __name__ == "__main__":
    title = main()
try:    
    title.run(1)
except ConnectionError as e:
    print(e)

scraping the 1 page...
['https://www.7234.cn/news/53968', 'https://www.7234.cn/abroad/53969', 'https://www.7234.cn/news/53967', 'https://www.7234.cn/news/53963', 'https://www.7234.cn/news/53966', 'https://www.7234.cn/news/53964', 'https://www.7234.cn/news/53962', 'https://www.7234.cn/news/53970', 'https://www.7234.cn/news/53965', 'https://www.7234.cn/news/53961']
the link is: https://www.7234.cn/news/53968
the link is: https://www.7234.cn/abroad/53969
the link is: https://www.7234.cn/news/53967
the link is: https://www.7234.cn/news/53963
the link is: https://www.7234.cn/news/53966
the link is: https://www.7234.cn/news/53964
the link is: https://www.7234.cn/news/53962
the link is: https://www.7234.cn/news/53970
the link is: https://www.7234.cn/news/53965
the link is: https://www.7234.cn/news/53961
scraping the 2 page...
['https://www.7234.cn/news/53959', 'https://www.7234.cn/news/53958', 'https://www.7234.cn/news/53954', 'https://www.7234.cn/news/53950', 'https://www.7234.cn/news/53949'

In [ ]:
os.mkdir("H:\Desktop\wang_ruo_yu_")

In [ ]:
photo = "https://i1.7234.cn/system/redactor_assets/pictures_3/000/148/183/148183/2018/105ac507921e3ab9f80f50e9c70fac6a.jpeg"
urlretrieve(photo, "H:Desktop\wang")

In [ ]:
str = "wang:ruo!yu?,"
pattern = re.compile(r'[\!,\?,\:]')
rep = re.sub(pattern,"_",str)
print(rep)

In [ ]:
https://i1.7234.cn/system/redactor_assets/pictures_3/000/148/104/148104/2018/8a895cc5326bfccc2f138b67a66fe316